In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from pathlib import Path
import os
import csv

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using:", device)

✅ Using: cuda


In [2]:
# Dataset paths
train_dir = r"D:\My Projects\Plant_Disease_Prediction_Custom_CNN\dataset_split\train"
val_dir = r"D:\My Projects\Plant_Disease_Prediction_Custom_CNN\dataset_split\val"
test_dir = r"D:\My Projects\Plant_Disease_Prediction_Custom_CNN\dataset_split\test"

# Class names
classes = sorted([d.name for d in Path(train_dir).iterdir()])
num_classes = len(classes)

# Normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_test_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transforms)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

In [3]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.3)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = CustomCNN(num_classes).to(device)

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

d:\Software\Anaconda\envs\pytorch-gpu\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [5]:
def save_checkpoint(model, optimizer, epoch, path="checkpoints"):
    os.makedirs(path, exist_ok=True)
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict()
    }, f"{path}/checkpoint_epoch_{epoch}.pth")

def load_latest_checkpoint(model, optimizer, path="checkpoints"):
    if not os.path.exists(path):
        return 0
    checkpoints = [f for f in os.listdir(path) if f.startswith("checkpoint_epoch_")]
    if not checkpoints:
        return 0
    latest = max(checkpoints, key=lambda x: int(x.split("_")[-1].split(".")[0]))
    checkpoint = torch.load(os.path.join(path, latest))
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    print(f"🔁 Resumed from {latest}")
    return checkpoint["epoch"]

In [6]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=15):
    start_epoch = load_latest_checkpoint(model, optimizer)
    best_val_acc = 0.0

    os.makedirs("logs", exist_ok=True)
    log_path = "logs/training_log.csv"
    if start_epoch == 0:
        with open(log_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["epoch", "train_loss", "train_acc", "val_loss", "val_acc"])

    for epoch in range(start_epoch, epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / len(train_loader)
        train_acc = correct / total

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total
        scheduler.step(val_loss)

        print(f"📘 Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

        # Save to CSV
        with open(log_path, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([epoch + 1, train_loss, train_acc, val_loss, val_acc])

        save_checkpoint(model, optimizer, epoch + 1)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_custom_cnn.pth")

    print(f"\n✅ Best Validation Accuracy: {best_val_acc:.4f}")

In [7]:
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=5)

📘 Epoch 1/5 | Train Loss: 0.7739, Acc: 0.7582 | Val Loss: 0.5601, Acc: 0.8113
📘 Epoch 2/5 | Train Loss: 0.4046, Acc: 0.8704 | Val Loss: 0.5468, Acc: 0.8088
📘 Epoch 3/5 | Train Loss: 0.3040, Acc: 0.9017 | Val Loss: 0.2480, Acc: 0.9190
📘 Epoch 4/5 | Train Loss: 0.2547, Acc: 0.9166 | Val Loss: 0.2075, Acc: 0.9296
📘 Epoch 5/5 | Train Loss: 0.2168, Acc: 0.9303 | Val Loss: 0.1807, Acc: 0.9389

✅ Best Validation Accuracy: 0.9389


In [8]:
def evaluate(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    test_acc = correct / total
    print(f"🎯 Test Accuracy: {test_acc:.4f}")

# Load and evaluate best model
model.load_state_dict(torch.load("best_custom_cnn.pth"))
evaluate(model, test_loader)


C:\Users\coder\AppData\Local\Temp\ipykernel_12736\3244498227.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_custom_cnn.pth"))


🎯 Test Accuracy: 0.9388
